# Style Reconstruction

In [ ]:
import torch
import PIL.Image as Image
import matplotlib.pyplot as plt
from visualization.visualize import feature_visualization , heat_map
from models.vgg import VggFeatureExtractor
from utils.image_utils import preprocess_image
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [ ]:
image_path = r"data\content\wallhaven-9d9g38.jpg"
imge = Image.open(image_path).convert("RGB")

In [ ]:
plt.figure(figsize=(5,5))
plt.axis('off')
plt.imshow(imge)
# plt.title('style image')
plt.show()

#### Step-1: import image and get the features maps of the respective layers [0 , 5 , 10 , 19 , 28]

In [ ]:
model = VggFeatureExtractor(target_layer=1)

with torch.no_grad():
    img = preprocess_image(image_path)
    features = model(img)

In [ ]:
feature_visualization(features)

In [ ]:
layers = [0 , 5 , 10 , 19 , 28]

In [ ]:
style_layers = {
    layer: VggFeatureExtractor(layer) for layer in layers
}

In [ ]:
style_layers

In [ ]:
# dict = {'l':2 , 't':5}
# for (i, j) in dict.items():
#     print(j)

In [ ]:
style_features = {}
img = preprocess_image(image_path)

for layer , model in style_layers.items():
    with torch.no_grad():
        style_features[layer] = model(img)

In [ ]:
for layer , feature in style_features.items():
    txt = f"layer : {layer} , dimensions : {feature.shape}"
    print(txt)
    # feature_visualization( feature , layer_name = txt)

#### Step-2: Calculate the gram matrix of the feature maps

In [ ]:
mat = torch.rand((1, 3, 2, 2))
mat

In [ ]:
B ,C , H ,W = mat.size()
new = mat.reshape( B , C , -1)
new

In [ ]:
trans = new.transpose(1, 2)
trans

In [ ]:
mul = torch.bmm(new , trans)
mul

In [ ]:
# ( B , C , H , W)
def _gram_matrix(features):
    B , C , H , W = features.size()
    features = features.view( B , C , H * W) # ( B , C , M) dim -3
    gram = torch.bmm(features , features.transpose(1, 2)) # ( C * M ) , ( M * C)
    return gram.div( C * H * W) # (B , C , C)

In [ ]:
style_gram_features = {}

for layer , features in style_features.items():
    with torch.no_grad():
        style_gram_features[layer] = _gram_matrix(features)

In [ ]:
heat_map(style_gram_features[layers[0]] , layer_name= f"Layer : {layers[0]}")

In [ ]:
style_weights = {layer : 1/5 for layer in layers}
style_weights

In [ ]:
def style_loss( original_gram , style_gram):
    return F.mse_loss(original_gram - style_gram)

In [ ]:
generated = torch.randn_like(img, requires_grad=True)
generated.shape

In [ ]:
img_gen = generated.squeeze(0).detach().cpu().numpy()
img_gen = np.transpose(img_gen, (1, 2, 0))
plt.imshow(img_gen)
plt.show()

In [ ]:
optimizer = optim.Adam([generated] ,lr= 0.01)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

In [ ]:
iterations = 100

def _total_loss(generated , style_gram_features):
    generated_style_fetures = 


def train():
    for i in range(iterations):
        total_loss = 